In [ ]:
import tensorflow as tf  # Import TensorFlow library for machine learning tasks
import numpy as np      # Import NumPy for numerical operations
import matplotlib.pyplot as plt  # Import Matplotlib for data visualization
import pickle           # Import Pickle for saving and loading data
import shutil           # Import shutil for file operations
import math             # Import math for mathematical operations
import pandas as pd     # Import Pandas for data manipulation
import gc               # Import gc for garbage collection
import os               # Import os for interacting with the operating system


# Model

In [ ]:
DEBUG = False

In [ ]:
class transformer_block(tf.keras.layers.Layer):
    def __init__(self, dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=dim//num_heads)
        self.ffn = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(feed_forward_dim, activation="relu"),
                tf.keras.layers.Dense(dim),
            ]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.supports_masking = True

    def call(self, inputs, training, mask):
        att_mask = tf.expand_dims(mask, axis=-1)
        att_mask = tf.repeat(att_mask, repeats=tf.shape(att_mask)[1], axis=-1)

        attn_output = self.att(inputs, inputs, attention_mask = att_mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class positional_encoding_layer(tf.keras.layers.Layer):
    def __init__(self, num_vocab=5, maxlen=500, hidden_dim=384):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.pos_emb = self.positional_encoding(maxlen-1, hidden_dim)
        self.supports_masking = True

    def call(self, x):
        maxlen = tf.shape(x)[-2]
        x = tf.math.multiply(x, tf.math.sqrt(tf.cast(self.hidden_dim, tf.float32)))
        return x + self.pos_emb[:maxlen, :]

    def positional_encoding(self, maxlen, hidden_dim):
        depth = hidden_dim/2
        positions = tf.range(maxlen, dtype = tf.float32)[..., tf.newaxis]
        depths = tf.range(depth, dtype = tf.float32)[np.newaxis, :]/depth
        angle_rates = tf.math.divide(1, tf.math.pow(tf.cast(10000, tf.float32), depths))
        angle_rads = tf.linalg.matmul(positions, angle_rates)
        pos_encoding = tf.concat(
          [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
          axis=-1)
        return pos_encoding

In [ ]:
X_max_len = 457
num_vocab = 5

def get_model(hidden_dim = 384, max_len = 206):
    inp = tf.keras.Input([None])
    x = inp

    x = tf.keras.layers.Embedding(num_vocab, hidden_dim, mask_zero=True)(x)
    x = positional_encoding_layer(num_vocab=num_vocab, maxlen=500, hidden_dim=hidden_dim)(x)

    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)

    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)

    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)
    x = transformer_block(hidden_dim, 6, hidden_dim*4)(x)

    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(2)(x)

    model = tf.keras.Model(inp, x)
    return model

In [ ]:
test_sequences_df = pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/test_sequences.csv')
test_sequences_df.head()

In [ ]:
test_sequences = test_sequences_df.sequence.to_numpy()
encoding_dict = {'A':1, 'C': 2, 'G': 3, 'U': 4}
encoding_dict

In [ ]:
max_len = 457 
test_sequences_encoded = []
for seq in test_sequences:
    test_sequences_encoded.append(
        np.concatenate([np.asarray([encoding_dict[x] for x in seq]), np.zeros((max_len - len(seq)))]).astype(np.float32))

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices(test_sequences_encoded)
batch_size = 256
if DEBUG:
    test_ds = test_ds.take(8)
    batch_size = 2
#test_ds = test_ds.take(10000)

test_ds = test_ds.padded_batch(batch_size, padding_values=(0.0), padded_shapes=([max_len]), drop_remainder=False)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)
batch = next(iter(test_ds))
batch.shape

In [ ]:
model = get_model(hidden_dim = 192,max_len = max_len)
model.load_weights('/kaggle/input/srrf-transformer-tpu-training/weights/model_epoch_74.h5')
model(batch)
model.summary()

In [ ]:
preds = model.predict(test_ds)

In [ ]:
preds_processed = []
for i, pred in enumerate(preds):
    preds_processed.append(pred[:len(test_sequences[i])])
concat_preds = np.concatenate(preds_processed)

In [ ]:
submission = pd.DataFrame({'id':np.arange(0, len(concat_preds), 1), 'reactivity_DMS_MaP':concat_preds[:,1], 'reactivity_2A3_MaP':concat_preds[:,0]})
submission.to_csv('submission.csv', index=False)
submission.head()